In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0


In [2]:
%%bash

pip freeze

aiohttp==3.8.3
aiorwlock==1.3.0
aiosignal==1.2.0
alembic==1.8.1
altair==4.2.0
anyio==3.6.1
appdirs==1.4.4
appnope==0.1.3
astroid==2.12.10
asttokens==2.0.8
async-generator==1.10
async-timeout==4.0.2
attrs==22.1.0
audioread==3.0.0
Authlib==1.2.0
azapi==3.0.8
azure-ai-formrecognizer==3.2.0b2
azure-common==1.1.28
azure-core==1.25.1
backcall==0.2.0
backoff==1.11.1
beautifulsoup4==4.11.1
beir==1.0.1
black==22.6.0
bleach==5.0.1
blinker==1.5
blobfile==2.0.1
brotlipy @ file:///Users/runner/miniforge3/conda-bld/brotlipy_1648854303569/work
bs4==0.0.1
cachetools==5.2.0
cattrs==22.1.0
certifi==2022.9.24
cffi @ file:///Users/runner/miniforge3/conda-bld/cffi_1656783163977/work
cfgv==3.3.1
charset-normalizer==2.0.12
ci-sdr==0.0.2
click==8.0.4
cloudpickle==2.2.0
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1655412516417/work
coloredlogs==15.0.1
commonmark==0.9.1
conda==22.9.0
conda-package-handling @ file:///Users/runner/miniforge3/conda-bld/conda-package-handling_1649385138064

In [3]:
from haystack.document_stores import InMemoryDocumentStore


# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

# Connect to Elasticsearch
document_store = InMemoryDocumentStore(
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    use_bm25=True
)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_by="word",
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(
    filename="data/nq_dev_subset_v2_short.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Updating BM25 representation...: 100%|██████████| 16/16 [00:00<00:00, 27526.20 docs/s]


In [5]:
from haystack.nodes import BM25Retriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

retriever = BM25Retriever(document_store=document_store)
reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [6]:
from haystack.schema import MultiLabel

eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)
eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 5}})
eval_result.save("gen_local")

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.93s/ Batches]


In [7]:
from haystack.schema import EvaluationResult

saved_eval_result = EvaluationResult.load(load_dir="gen_local")
metrics = saved_eval_result.calculate_metrics()

print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

SyntaxError: invalid syntax (<unknown>, line 0)